# Data Preprocessing

## Environment Setup

In [1]:
%pip install -r '../requirements.txt'

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import glob
import torch
import pandas as pd
from datetime import datetime
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

/Users/brbharad/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/brbharad/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/brbharad/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/brbharad/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/brbharad/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/brbharad/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/brbharad/nltk_data...
[nltk_data]    |   Package movie_review

True

In [4]:
# check if torch finds cuda
torch.cuda.is_available()

False

In [ ]:
# CONSTANTS
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FILENAME = '../Dataset/scraper/raw/rawdata_1731148570.344731.json'

## Data Processing

In [6]:
summarizer = pipeline("summarization", device=DEVICE)
def get_summary(text):
    summary = summarizer(text, truncation=True, max_length=300, min_length=50, do_sample=True, temperature=0.3)
    return summary[0]['summary_text']

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/brbharad/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def get_vader_score(text):
    if type(text) != str or len(text) == 0:    return 0
    text = text.strip()
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    return sentiment["compound"]

In [ ]:
# %%script false --no-raise-error
with open(FILENAME, 'r') as f:
    data = json.load(f)
article_df = pd.DataFrame(data)
article_df['datetime'] = pd.to_datetime(article_df['datetime'])
article_df.dropna(subset=['datetime'], inplace=True)
article_df.head()

In [ ]:
# %%script false --no-raise-error
article_df['summary'] = article_df['text'].apply(get_summary)
article_df['summary_vader'] = article_df['summary'].apply(get_vader_score)
article_df['description_vader'] = article_df['description'].apply(get_vader_score)

In [ ]:
# %%script false --no-raise-error
article_df.drop(columns=['metadata', 'text'], inplace=True)
article_df.sort_values('datetime', inplace=True)
article_df.to_excel(FILENAME.replace('raw', 'preprocessed').replace('.json', '.xlsx'), index=False)
article_df.head()

In [ ]:
# Read each preprocessed file and concatenate them into a single DataFrame
excel_files = glob.glob("../Dataset/scraper/processed/articledf_*.xlsx")
df_list = [pd.read_excel(file, parse_dates=['datetime']) for file in excel_files]
article_df = pd.concat(df_list, ignore_index=True)
article_df.drop_duplicates(subset=['url'])
article_df.sort_values('datetime', inplace=True)
article_df.to_excel("../Dataset/scraper/market_article_df.xlsx", index=False)